# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an **African restaurant** in **Toronto**, Canada.

Since African foods are not very popular, our strategy will be to bring them near to other restaurants; so that most new customers shall be discovering African meals while going to have a launch. Another thing is that when restaurants are gathered, people tend to visit the neighborhood at launch time, thus giving them the opportunity to discover African restaurants. We will try to detect **locations with high density of restaurants**. We are also particularly interested in **areas with no African restaurants in vicinity**. We would also prefer locations **as close to city center as possible**, assuming that those conditions are met.

We will use our data science powers to generate a few most promising neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decision are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to African restaurants in the neighborhood, if any
* distance of neighborhood from city center

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **OpenCage reverse Geocoder API**
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Toronto center will be obtained using **OpenCage Geocoder API** of well known Toronto location (Yonge–Dundas Square)

### Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 12x12 killometers centered around Toronto city center.

Let's first find the latitude & longitude of Toronto city center, using specific, well known address and OpenCage Geocoder API.

In [1]:
!pip install opencage

api_key = 'eb991265a04341089b35ba8971200001'

In [2]:
from opencage.geocoder import OpenCageGeocode
from pprint import pprint
import requests


def get_coordinates(api_key, address, verbose=False):
    try:
        geocoder = OpenCageGeocode(api_key)
        if verbose:
            print('OpenCage Geocoder API JSON result =>', geocoder)
        result = geocoder.geocode(address)
        lat = result[0]['geometry']['lat']
        lon = result[0]['geometry']['lng']
        return [lat, lon]
    except:
        return [None, None]

address = 'Yonge–Dundas Square'
toronto_center = get_coordinates(api_key, address)
print('Coordinate of {}: {}'.format(address, toronto_center))
#geocoder = OpenCageGeocode(api_key)

#results = geocoder.reverse_geocode(times_square_lat, times_square_lon)

#town = geocoder.geocode('Yonge–Dundas Square')

Coordinate of Yonge–Dundas Square: [43.6560992, -79.3801563]


Now let's create a grid of area candidates, equally spaced, centered around city center and within ~6km from Yonge–Dundas Square. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in  meters).

In [3]:
!conda install -c conda-forge shapely --yes
import shapely.geometry

#!pip install pyproj
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Yonge–Dundas Square longitude={}, latitude={}'.format(toronto_center[1], toronto_center[0]))
x, y = lonlat_to_xy(toronto_center[1], toronto_center[0])
print('Yonge–Dundas Square UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Yonge–Dundas Square longitude={}, latitude={}'.format(lo, la))

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs:
    - shapely


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    basemap-1.2.0              |py36h673bf1a_1001        15.1 MB  conda-forge
    conda-4.6.7                |           py36_0         869 KB  conda-forge
    geos-3.7.1                 |    hf484d3e_1000         1.6 MB  conda-forge
    openssl-1.1.1b             |       h14c3975_0         4.0 MB  conda-forge
    shapely-1.6.4              |py36h092830f_1002         330 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        21.9 MB

The following NEW packages will be INSTALLED:

  shapely            conda-forge/linux-64::shapely-1.6.4-py36h092830f_1002

The following packages will be UPDATED:

  basemap            anaconda::base

Let's create a **hexagonal grid of cells**: we offset every other row, and adjust vertical row spacing so that **every cell center is equally distant from all it's neighbors**.

In [4]:
toronto_center_x, toronto_center_y = lonlat_to_xy(toronto_center[1], toronto_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = toronto_center_x - 6000
x_step = 600
y_min = toronto_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(toronto_center_x, toronto_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


Let's visualize the data we have so far: city center location and candidate neighborhood centers:

In [5]:
import folium

In [6]:
map_toronto = folium.Map(location=toronto_center, zoom_start=13)
folium.Marker(toronto_center, popup='Yonge–Dundas Square').add_to(map_toronto)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_berlin) 
    folium.Circle([lat, lon], radius=200, color='green', fill=False).add_to(map_toronto)
    #folium.Marker([lat, lon]).add_to(map_berlin)
map_toronto

OK, we now have the coordinates of centers of neighborhoods/areas to be evaluated, equally spaced (distance from every point to it's neighbors is exactly the same) and within ~6km from Yonge–Dundas Square. 

Let's now use OpenCage Geocoder API to get approximate addresses of those locations.

In [7]:
def get_address(api_key, latitude, longitude, verbose=False):
    try:
        geocoder = OpenCageGeocode(api_key)
        results = geocoder.reverse_geocode(latitude, longitude)
        if verbose:
            print('Google Maps API JSON result =>', results)
        address = results[0]['formatted']
        return address
    except:
        return None

addr = get_address(api_key, toronto_center[0], toronto_center[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(toronto_center[0], toronto_center[1], addr))

Reverse geocoding check
-----------------------
Address of [43.6560992, -79.3801563] is: 20 Yonge-Dundas Square, Toronto, ON M5B 0A1, Canada


In [8]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(api_key, lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    address = address.replace(', Canada', '') # We don't need country part of address
    addresses.append(address)
    print(' .', end='')
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [9]:
addresses[150:170]

['152 Douglas Drive, Toronto, ON M4T 1C7',
 '13 Errington Avenue, Toronto, ON M4T 1C9',
 '105 Hudson Drive, Toronto, ON M4T 1C9',
 'Royal Canadian Yacht Club, Chippewa Avenue, Toronto, ON M5J 1X9',
 'Inner Harbour, Chippewa Avenue, Toronto, ON M5J 1X9',
 'Inner Harbour, Chippewa Avenue, Toronto, ON M5J 1X9',
 'Inner Harbour, Chippewa Avenue, Toronto, ON M5J 1X9',
 'Martin Goodman Trail, 4, Old Toronto, Toronto, ON M5E',
 '29 Queens Quay East, Toronto, ON M5E',
 '53 Freeland Street, Toronto, ON M5E',
 '60 Colborne Street, Toronto, ON M5E 1E3',
 'GOP Building, 107 Church Street, Toronto, ON M5B 1Z2',
 '98 Mutual Street, Toronto, ON M5B 2B2',
 'T.O. Condo, 117 Gerrard Street East, Toronto, ON M5B 1G7',
 '433 Jarvis Street, Toronto, ON M4Y 2G6',
 '10 Wellesley Place, Toronto, ON M4Y 2G8',
 'Greenwin Square, Selby Street, Toronto, ON M4X 1W4',
 '22 Elm Avenue, Toronto, ON M4W 3J6',
 '166 Crescent Road, Toronto, ON M4W 1R1',
 '32 Highland Avenue, Toronto, ON M4W 1R1']

In [10]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,"Portlands Energy Centre, 470 Unwin Avenue, Tor...",43.648823,-79.329586,-5.311959e+06,1.050134e+07,5992.495307
1,"Portlands Energy Centre, 470 Unwin Avenue, Tor...",43.652549,-79.330147,-5.311359e+06,1.050134e+07,5840.376700
2,Toronto Transit Commission - Lakeshore Complex...,43.656275,-79.330708,-5.310759e+06,1.050134e+07,5747.173218
3,"2 Rushbrooke Avenue, Toronto, ON M4M 1L4",43.660001,-79.331268,-5.310159e+06,1.050134e+07,5715.767665
4,"18 Curzon Street, Toronto, ON M4M 1L7",43.663727,-79.331829,-5.309559e+06,1.050134e+07,5747.173218
5,"Leslieville Junior Public School, 254 Leslie S...",43.667453,-79.332391,-5.308959e+06,1.050134e+07,5840.376700
6,"413 Leslie Street, Toronto, ON M4M 2A1",43.671180,-79.332952,-5.308359e+06,1.050134e+07,5992.495307
7,"Martin Goodman Trail, Old Toronto, Toronto, ON...",43.642883,-79.333188,-5.312859e+06,1.050186e+07,5855.766389
8,Richard L. Hearn Thermal Generating Station (c...,43.646608,-79.333749,-5.312259e+06,1.050186e+07,5604.462508
9,"109 Basin Street, Toronto, ON M5A 1A6",43.650333,-79.334310,-5.311659e+06,1.050186e+07,5408.326913


### Foursquare
Now that we have our location candidates, let's use Foursquare API to get info on restaurants in each neighborhood.

We're interested in venues in 'food' category, but only those that are proper restaurants - coffe shops, pizza places, bakeries etc. are not direct competitors so we don't care about those. So we will include in out list only venues that have 'restaurant' in category name, and we'll make sure to detect and include all the subcategories of specific 'African restaurant' category, as we need info on African restaurants in the neighborhood.

In [11]:
# Foursquare Credentials

fsq_client_id = 'FZWQ1YEP4VG5V3WGVK3OD4LXXYYUNRLHN1BN05YJXX440MH4' # your Foursquare ID
fsq_client_secret = 'EYLRON0METXI1FQTGKPNH4V3T1UMFA5VQELHVS20RSHD5EBK' # your Foursquare Secret
#version = '20180605' # Foursquare API version

print('My credentails:')
print('CLIENT_ID: ' + fsq_client_id)
print('CLIENT_SECRET:' + fsq_client_secret)

My credentails:
CLIENT_ID: FZWQ1YEP4VG5V3WGVK3OD4LXXYYUNRLHN1BN05YJXX440MH4
CLIENT_SECRET:EYLRON0METXI1FQTGKPNH4V3T1UMFA5VQELHVS20RSHD5EBK


In [12]:
# Category IDs corresponding to African restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

african_restaurant_categories = ['4bf58dd8d48988d1c8941735', '4bf58dd8d48988d10a941735']

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Canada', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = ['rien', 'rien']
    return venues

#get_venues_near_location(lat, lon, food_category, fsq_client_id, fsq_client_secret)

In [13]:
# Let's now go over our neighborhood locations and get nearby restaurants; we'll also maintain a dictionary of all found restaurants and all found italian restaurants

#import pickle

def get_restaurants(lats, lons):
    restaurants = {}
    african_restaurants = {}
    location_restaurants = []
    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, fsq_client_id, fsq_client_secret, radius=350, limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_african = is_restaurant(venue_categories, specific_filter=african_restaurant_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_african, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_african:
                    african_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, african_restaurants, location_restaurants

restaurants, african_restaurants, location_restaurants = get_restaurants(latitudes, longitudes)





Obtaining venues around candidate locations: . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [14]:
import numpy as np

print('Total number of restaurants:', len(restaurants))
print('Total number of African restaurants:', len(african_restaurants))
print('Percentage of African restaurants: {:.2f}%'.format(len(african_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 1216
Total number of African restaurants: 11
Percentage of African restaurants: 0.90%
Average number of restaurants in neighborhood: 5.233516483516484


Let's now see all the collected restaurants on map, and let's also show African restaurants in different color.

In [15]:
map_toronto = folium.Map(location=toronto_center, zoom_start=13)
folium.Marker(toronto_center, popup='Yonge-Dundas Square').add_to(map_toronto)
for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_african = res[6]
    color = 'red' if is_african else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_toronto)
map_toronto

Looking good. So now we have all the restaurants in area within few kilometers from Yonge-Dundas Square, and we know which ones are African restaurants! We also know which restaurants exactly are in vicinity of every neighborhood candidate center.

This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new African restaurant!

## Methodology <a name="methodology"></a>

In this project we will direct our efforts on detecting areas of Toronto that have high restaurant density, particularly those with low number of African restaurants. We will limit our analysis to area ~6km around city center.

In first step we have collected the required **data: location and type (category) of every restaurant within 6km from Toronto center** (Yonge-Dundas Square). We have also **identified African restaurants** (according to Foursquare categorization).

Second step in our analysis will be calculation and exploration of '**restaurant density**' across different areas of Toronto.

In third and final step we will create **clusters of locations that meet some basic requirements** established in discussion with stakeholders: we will take into consideration locations with **at least ten restaurants in radius of 250 meters**, and we want locations **without African restaurants in radius of 500 meters**. We will present map of all such locations but also create clusters (using **k-means clustering**) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.

## Analysis <a name="analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First let's count the **number of restaurants in every area candidate**:

In [16]:
location_restaurants_count = [len(res) for res in location_restaurants]

df_locations['Restaurants in area'] = location_restaurants_count

print('Average number of restaurants in every area with radius=300m:', np.array(location_restaurants_count).mean())

df_locations.head(10)

Average number of restaurants in every area with radius=300m: 5.233516483516484


,Address,Latitude,Longitude,X,Y,Distance from center,Restaurants in area
0,"Portlands Energy Centre, 470 Unwin Avenue, Tor...",43.648823,-79.329586,-5.311959e+06,1.050134e+07,5992.495307,0
1,"Portlands Energy Centre, 470 Unwin Avenue, Tor...",43.652549,-79.330147,-5.311359e+06,1.050134e+07,5840.376700,0
2,Toronto Transit Commission - Lakeshore Complex...,43.656275,-79.330708,-5.310759e+06,1.050134e+07,5747.173218,0
3,"2 Rushbrooke Avenue, Toronto, ON M4M 1L4",43.660001,-79.331268,-5.310159e+06,1.050134e+07,5715.767665,1
4,"18 Curzon Street, Toronto, ON M4M 1L7",43.663727,-79.331829,-5.309559e+06,1.050134e+07,5747.173218,9
5,"Leslieville Junior Public School, 254 Leslie S...",43.667453,-79.332391,-5.308959e+06,1.050134e+07,5840.376700,0
6,"413 Leslie Street, Toronto, ON M4M 2A1",43.671180,-79.332952,-5.308359e+06,1.050134e+07,5992.495307,0
7,"Martin Goodman Trail, Old Toronto, Toronto, ON...",43.642883,-79.333188,-5.312859e+06,1.050186e+07,5855.766389,0
8,Richard L. Hearn Thermal Generating Station (c...,43.646608,-79.333749,-5.312259e+06,1.050186e+07,5604.462508,0
9,"109 Basin Street, Toronto, ON M5A 1A6",43.650333,-79.334310,-5.311659e+06,1.050186e+07,5408.326913,0


OK, now let's calculate the **distance to nearest African restaurant from every area candidate center** (not only those within 300m - we want distance to closest one, regardless of how distant it is).

In [17]:
distances_to_african_restaurant = []

for area_x, area_y in zip(xs, ys):
    min_distance = 10000
    for res in african_restaurants.values():
        res_x = res[7]
        res_y = res[8]
        d = calc_xy_distance(area_x, area_y, res_x, res_y)
        if d<min_distance:
            min_distance = d
    distances_to_african_restaurant.append(min_distance)

df_locations['Distance to African restaurant'] = distances_to_african_restaurant

OK. Now let's calculate one more important thing for each location candidate: **number of restaurants in vicinity** (we'll use radius of **250 meters**)

In [18]:
Resto_within_250 = []

for index, row in df_locations.iterrows():
    count = 0
    for res in restaurants.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(row['X'], row['Y'], res_x, res_y)
        if d<=250:
            count += 1
    Resto_within_250.append(count)

df_locations['Number of Restaurant whitin 250'] = Resto_within_250


In [20]:
df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center,Restaurants in area,Distance to African restaurant,Number of Restaurant whitin 250
0,"Portlands Energy Centre, 470 Unwin Avenue, Tor...",43.648823,-79.329586,-5.311959e+06,1.050134e+07,5992.495307,0,4948.095439,0
1,"Portlands Energy Centre, 470 Unwin Avenue, Tor...",43.652549,-79.330147,-5.311359e+06,1.050134e+07,5840.376700,0,4819.872439,0
2,Toronto Transit Commission - Lakeshore Complex...,43.656275,-79.330708,-5.310759e+06,1.050134e+07,5747.173218,0,4764.314451,0
3,"2 Rushbrooke Avenue, Toronto, ON M4M 1L4",43.660001,-79.331268,-5.310159e+06,1.050134e+07,5715.767665,1,4783.953809,1
4,"18 Curzon Street, Toronto, ON M4M 1L7",43.663727,-79.331829,-5.309559e+06,1.050134e+07,5747.173218,9,4877.882317,5
5,"Leslieville Junior Public School, 254 Leslie S...",43.667453,-79.332391,-5.308959e+06,1.050134e+07,5840.376700,0,5041.949797,0
6,"413 Leslie Street, Toronto, ON M4M 2A1",43.671180,-79.332952,-5.308359e+06,1.050134e+07,5992.495307,0,5269.609057,0
7,"Martin Goodman Trail, Old Toronto, Toronto, ON...",43.642883,-79.333188,-5.312859e+06,1.050186e+07,5855.766389,0,4799.312297,0
8,Richard L. Hearn Thermal Generating Station (c...,43.646608,-79.333749,-5.312259e+06,1.050186e+07,5604.462508,0,4549.826412,0
9,"109 Basin Street, Toronto, ON M5A 1A6",43.650333,-79.334310,-5.311659e+06,1.050186e+07,5408.326913,0,4369.032185,0


Since African foods are not very popular, our strategy will be to bring them near to other restaurants; so that most new customers shall be discovering African meals while going to have a launch. Another thing is that when restaurants are gathered, people tend to visit the neighborhood at launch time, thus giving them the opportunity to discover African restaurants.
OK. Let us now **filter** those locations: we're interested only in **locations with at least 10 restaurants in radius of 250 meters**, and **no African restaurants in radius of 500 meters**. Another criteria will be **locations in a radius of 2Km from Yonge-Dundas Square**

In [20]:
print('Average distance to closest African restaurant from each area center:', df_locations['Distance to African restaurant'].mean())

Average distance to closest African restaurant from each area center: 2532.8634135961274


OK, so **on average African restaurant can be found within ~2500m** from every area center candidate.

In [23]:
good_res_count = np.array((df_locations['Number of Restaurant whitin 250']>=10))
print('Locations with at least 10 restaurants nearby:', good_res_count.sum())

good_afr_distance = np.array(df_locations['Distance to African restaurant']>=500)
print('Locations with no African restaurants within 500m:', good_afr_distance.sum())

good_center_distance = np.array(df_locations['Distance from center']<=2000)
print('Locations within 2Km from the city center:', good_center_distance.sum())

good_locations = np.logical_and(np.logical_and(good_res_count, good_afr_distance), good_center_distance)
print('Locations with the 3 conditions met:', good_locations.sum())

df_good_locations = df_locations[good_locations]

toronto_boroughs_url = 'https://raw.githubusercontent.com/jasonicarter/toronto-geojson/master/toronto_crs84.geojson'
toronto_boroughs = requests.get(toronto_boroughs_url).json()

def boroughs_style(feature):
    return { 'color': 'blue', 'fill': False }

Locations with at least 10 restaurants nearby: 29
Locations with no African restaurants within 500m: 343
Locations within 2Km from the city center: 42
Locations with the 3 conditions met: 7


Looking good. What we have now is a clear indication of zones with low number of restaurants in vicinity, and *no* African restaurants at all nearby.

Let us now **cluster** those locations to create **centers of zones containing good locations**. Those zones, their centers and addresses will be the final result of our analysis. 

In [24]:
restaurant_latlons = [[res[2], res[3]] for res in restaurants.values()]

african_latlons = [[res[2], res[3]] for res in african_restaurants.values()]

good_latitudes = df_good_locations['Latitude'].values
good_longitudes = df_good_locations['Longitude'].values

good_locations = [[lat, lon] for lat, lon in zip(good_latitudes, good_longitudes)]

In [25]:
from sklearn.cluster import KMeans
from folium import plugins
from folium.plugins import HeatMap

number_of_clusters = 4

good_xys = df_good_locations[['X', 'Y']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)

cluster_centers = [xy_to_lonlat(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

map_toronto = folium.Map(location=toronto_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_toronto)
HeatMap(restaurant_latlons).add_to(map_toronto)
folium.Circle(toronto_center, radius=2000, color='white', fill=True, fill_opacity=0.4).add_to(map_toronto)
folium.Marker(toronto_center).add_to(map_toronto)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=True, fill_opacity=0.25).add_to(map_toronto) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_toronto)
#folium.GeoJson(toronto_boroughs, style_function=boroughs_style, name='geojson').add_to(map_toronto)
map_toronto

In [26]:
candidate_area_addresses = []
print('==============================================================')
print('Addresses of centers of areas recommended for further analysis')
print('==============================================================\n')
for lon, lat in cluster_centers:
    addr = get_address(api_key, lat, lon).replace(', Canada', '')
    candidate_area_addresses.append(addr)    
    x, y = lonlat_to_xy(lon, lat)
    d = calc_xy_distance(x, y, toronto_center_x, toronto_center_y)
    print('{}{} => {:.1f}km from Yonge-Dundas Square'.format(addr, ' '*(50-len(addr)), d/1000))

Addresses of centers of areas recommended for further analysis

Commerce Court, Traders Bank, 67 Yonge Street, Toronto, ON M5E 1J8 => 1.2km from Yonge-Dundas Square
12 Maitland Street, Toronto, ON M4Y 1X9            => 1.4km from Yonge-Dundas Square
38 D'Arcy Street, Toronto, ON M5T 1Y5              => 1.6km from Yonge-Dundas Square
Qwest Condo, Richmond Street West, Toronto, ON M5V 1V6 => 1.4km from Yonge-Dundas Square


This concludes our analysis. We have created 4 addresses representing centers of zones containing locations with high number of restaurants and no African restaurants nearby, all zones being fairly close to city center (all less than 2km from Yonge-Dundas Square). Although zones are shown on map with a radius of ~500 meters (green circles), their shape is actually very irregular and their centers/addresses should be considered only as a starting point for exploring area neighborhoods in search for potential restaurant locations.

## Results and Discussion <a name="results"></a>

Our analysis shows that although there is a great number of restaurants in Toronto (~1200 in our initial area of interest which was 12x12km around Yonge-Dundas Square), there are pockets of low restaurant density fairly close to city center. 

After directing our attention to this more narrow area of interest we first created a dense grid of location candidates (spaced 100m appart); those locations were then filtered so that those with less than 10 restaurants in radius of 250m and those with an African restaurant closer than 500m were removed.

Those location candidates were then clustered to create zones of interest which contain greatest number of location candidates. Addresses of centers of those zones were also generated using OpenCage reverse Geocoder API to be used as markers/starting points for more detailed local analysis based on other factors.

Result of all this is 4 zones containing largest number of potential new restaurant locations based on number of and distance to existing venues - both restaurants in general and African restaurants particularly. This, of course, does not imply that those zones are actually optimal locations for a new restaurant! Purpose of this analysis was to only provide info on areas close to Toronto center but not crowded with existing restaurants (particularly African) - it is entirely possible that there is a very good reason for small number of restaurants in any of those areas, reasons which would make them unsuitable for a new restaurant regardless of lack of competition in the area. Recommended zones should therefore be considered only as a starting point for more detailed analysis which could eventually result in location which has not only no nearby competition but also other factors taken into account and all other relevant conditions met.

## Conclusion <a name="conclusion"></a>

Purpose of this project was to identify Toronto areas close to center with high number of restaurants (particularly with no African restaurants) in order to aid stakeholders in narrowing down the search for optimal location for a new African restaurant. By calculating restaurant density distribution from Foursquare data we have generated extensive collection of locations which satisfy some basic requirements regarding existing nearby restaurants. Clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations) and addresses of those zone centers were created to be used as starting points for final exploration by stakeholders.

Final decision on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.